In [153]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from string import printable
from typing import  List
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import BernoulliNB
from google.colab import drive

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [154]:
df = pd.read_csv('/content/train_spam.csv') #просто прочитать данные
df_test = pd.read_csv('/content/test_spam.csv')

In [155]:
df['text_type'].unique() #посмотреть на флаги спам\не спам

array(['ham', 'spam'], dtype=object)

In [156]:
df.head()

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


> Выглядит так, как будто датасет уже предобработан, потому что в первых строках все с маленькой буквы, нет специальных знаков, цифр или пунктуации

In [157]:
df.isna().sum() #датасет без пропусков :)

text_type    0
text         0
dtype: int64

In [158]:
df.groupby('text_type').count()['text'] #классы не очень сбалансированные

text_type
ham     11469
spam     4809
Name: text, dtype: int64

In [159]:
#разделим на тренировочную и валидационную выборку
train, val = train_test_split(df, test_size=0.3, random_state=42, stratify=df['text_type'])
#stratify нужен, чтобы классы с правильным распределением попали в выборки

> Для одного из учебных проектов я писала функцию предобработки твитов на английском, здесь она тоже частично подходит

In [160]:
lang = 'english'

stopwords_set = set(stopwords.words(lang))
stemmer = SnowballStemmer(lang)

def contains_only_latin_letters(s: str) -> bool:
    # Проверка, содержит ли слово только латинские буквы
    return not re.search(r'[^a-zA-Z]', s)

def is_emoji(s: str) -> bool:
    # Проверка, является ли слово смайликом
    return bool(re.search(r'[\(]+|[\)]+',s)) & (not bool(re.search(r'[^\(\)\!\"\#\$\%\&\\\'\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\\\]\^\_\`\{\|\}\~]',s)))

def is_hashtag(s: str) -> bool:
    # Проверка, является ли слово хэштегом
    res = False
    if s[0]=='#' and len(s)==1:
        res = True
    elif s[0]=='#' and contains_only_latin_letters(s[1:]):
        res = True
    return res

def is_used(s: str) -> bool:
    return ((contains_only_latin_letters(s) or is_emoji(s) or is_hashtag(s)) and s not in stopwords_set)

def custom_tokenizer(s: str) -> List[str]:
    # Кастомный токенайзер
    tk = TweetTokenizer()
    words = filter(is_used, tk.tokenize(s.lower()))
    stemmed = [stemmer.stem(w) for w in words]
    return (stemmed)

In [161]:
#теперь нужно применить токенизатор с написанной выше функцией к тренировочным и тестовым данным
vectorizer = CountVectorizer(tokenizer = custom_tokenizer )
X = vectorizer.fit_transform(train['text'])
y = vectorizer.transform(val['text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


> Так как у нас задача бинарной классификации, можно попробовать использовать логистическую регрессию в качестве бейзлайна, потом случайный лес и еще наивный байесовский классификатор (он раньше часто использовался для классификации спама)

In [162]:
#попробуем обучить логрег
clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(X, train['text_type'])
roc_auc_score(val['text_type'], clf.predict_proba(y)[:, 1])

0.9680445316888211

In [163]:
np.unique(clf.predict(y), return_counts=True)

(array(['ham', 'spam'], dtype=object), array([3615, 1269]))

> Качество уже достаточно высокое, и модель не выдает чисто константное предсказание, распределение классов чем-то напоминает исходный датасет

In [164]:
#попробуем другой токенайзер, который тоже часто используется для текстов
tfidf_vec = TfidfVectorizer(tokenizer = custom_tokenizer)
X_tf = tfidf_vec.fit_transform(train['text'])
y_tf = tfidf_vec.transform(val['text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [165]:
#снова обучим модель логистической регрессии
clf_tf = LogisticRegression(max_iter=200, random_state=42)
clf_tf.fit(X_tf, train['text_type'])
roc_auc_score(val['text_type'], clf_tf.predict_proba(y_tf)[:, 1])

0.9679396048184191

In [166]:
np.unique(clf_tf.predict(y_tf), return_counts=True)

(array(['ham', 'spam'], dtype=object), array([3678, 1206]))

> Качество понизилось, но это может быть связано со спецификой токенизатора, потому что он опирается не только на обучающую выборку

In [167]:
#теперь попробуем обучить Random Forest и подобрать к нему гиперпараметры

In [168]:
param_grid = {
    'n_estimators': [25, 50, 100],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [2, 4, 6],
    'max_leaf_nodes': [2, 4, 6],
}

In [169]:
random_search = RandomizedSearchCV(RandomForestClassifier(),
                                   param_grid)
random_search.fit(X, train['text_type']) #используем первый токенизатор
print(random_search.best_estimator_)

RandomForestClassifier(max_depth=2, max_features='log2', max_leaf_nodes=2,
                       n_estimators=25)


In [170]:
rf = RandomForestClassifier(n_estimators=50, max_depth=6, max_leaf_nodes=6)
rf.fit(X, train['text_type'])
roc_auc_score(val['text_type'], rf.predict_proba(y)[:, 1])

0.8973126436073254

In [171]:
np.unique(rf.predict(y), return_counts=True)

(array(['ham'], dtype=object), array([4884]))

> Эта модель очень плохо предсказывает спам, ее точно не стоит использовать

In [172]:
#обучим байесовский классификатор
bnb = BernoulliNB()
bnb.fit(X, train['text_type'])
roc_auc_score(val['text_type'], bnb.predict_proba(y)[:, 1])

0.9692365895504519

In [173]:
np.unique(bnb.predict(y), return_counts=True)

(array(['ham', 'spam'], dtype='<U4'), array([3646, 1238]))

> Как и требовалось ожидать, эта модель оказалась самой лучшей, все-таки не зря ее раньше часто использовали для классификации спама

In [174]:
bnb.classes_ #получается в итоговый датасет я запишу только вторую вероятность принадлежности к спаму

array(['ham', 'spam'], dtype='<U4')

In [175]:
#посчитаем скоры соответствия спаму с помощью байесовского классификатора для тестовой выборки и сохраним в csv
y_test = vectorizer.transform(df_test['text'])
df_test['score'] = bnb.predict_proba(y_test)[:, 1]

In [177]:
drive.mount('/content/drive', force_remount = True)
df_test.to_csv('/content/drive/My Drive/results.csv', index=False)

Mounted at /content/drive
